<a href="https://colab.research.google.com/github/zar373/Loan_Prediction/blob/main/Loan_Prediction_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.model_selection import StratifiedKFold

In [ ]:
trainm= pd.read_csv('/content/sample_data/train_modified.csv')
testm= pd.read_csv('/content/sample_data/test_modified.csv')

In [ ]:
trainm.drop('Unnamed: 0',axis=1,inplace=True)
testm.drop('Unnamed: 0',axis=1,inplace=True)

In [ ]:
trainm.columns

Index(['Credit_History', 'Loan_Status', 'logLoanAmount', 'Gender_Female',
       'Gender_Male', 'Married_No', 'Married_Yes', 'Dependents_3',
       'Dependents_0', 'Dependents_1', 'Dependents_2', 'Education_Graduate',
       'Education_Not Graduate', 'Self_Employed_No', 'Self_Employed_Yes',
       'Property_Area_Rural', 'Property_Area_Semiurban', 'Property_Area_Urban',
       'Total_Income', 'logTotalIncome', 'EMI', 'Balance_Income'],
      dtype='object')

In [ ]:
X= trainm.drop('Loan_Status', axis= 1)
y= trainm.Loan_Status

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.3)

In [ ]:
i= 1
kf= StratifiedKFold(n_splits= 5, random_state=1, shuffle= True)
for train_index, test_index in kf.split(X, y):
  print('\n{}  of kfold{}'.format(i, kf.n_splits))
  xtr, xvl= X.loc[train_index], X.loc[test_index]
  ytr, yvl= y[train_index], y[test_index]

  LogReg= LogisticRegression(random_state=1, max_iter= 200)
  LogReg.fit(xtr, ytr)
  pred_test= LogReg.predict(xvl)
  score= metrics.accuracy_score(yvl, pred_test)
  print('accuracy score', score)
  i +=1
  pred_test= LogReg.predict(testm)




1  of kfold5
accuracy score 0.7967479674796748

2  of kfold5
accuracy score 0.8373983739837398

3  of kfold5
accuracy score 0.6666666666666666

4  of kfold5
accuracy score 0.7804878048780488

5  of kfold5
accuracy score 0.7950819672131147


In [ ]:
from sklearn import tree

i= 1
kf= StratifiedKFold(n_splits= 5, random_state= 1, shuffle= True)
for train_index, test_index in kf.split(X, y):
  print("\n{} of fold {}".format(i, kf.n_splits))
  xtr, xvl= X.loc[train_index], X.loc[test_index]
  ytr, yvl= y[train_index], y[test_index]

  DecTre= tree.DecisionTreeClassifier(random_state= 1)
  DecTre.fit(xtr, ytr)
  pred_test= DecTre.predict(xvl)
  score= metrics.accuracy_score(yvl, pred_test)
  print('accuracy score', score)
  i +=1
  pred_test= DecTre.predict(testm)
  # print(pred_test)



1 of fold 5
accuracy score 0.7723577235772358
[1 1 1 1 1 0 1 0 1 1 1 0 1 1 1 1 1 1 0 1 1 0 1 1 1 0 1 1 1 0 1 1 1 1 1 1 1
 0 1 1 0 1 1 1 0 0 0 1 1 0 1 1 1 1 0 0 1 1 1 0 1 1 1 0 1 1 0 0 1 0 1 0 0 0
 0 1 0 1 1 0 1 1 0 0 0 1 1 1 1 1 1 0 0 1 0 1 0 1 1 1 1 0 1 1 1 1 0 1 1 0 1
 1 1 1 0 1 1 0 0 0 0 1 1 0 0 1 0 1 1 1 1 0 1 1 1 1 1 1 1 1 0 1 0 1 0 1 1 0
 1 1 0 1 1 0 1 1 1 1 1 0 0 0 1 0 1 1 0 1 0 1 1 0 1 0 0 1 0 0 1 1 0 0 1 1 1
 1 0 1 1 1 0 1 0 0 1 1 0 1 0 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 0 1 1 1 1 1 1
 0 1 0 1 1 1 1 0 1 1 1 1 1 1 0 1 1 0 1 0 1 0 1 0 1 1 0 1 0 1 1 1 1 0 1 0 1
 1 1 1 1 0 0 1 0 1 0 1 1 0 0 0 0 1 0 1 1 1 1 1 1 1 1 1 0 0 1 1 1 1 1 0 1 0
 0 1 1 0 1 0 0 1 1 0 1 1 1 1 1 0 1 1 0 1 1 0 1 1 0 0 1 1 0 0 1 1 1 1 1 1 1
 1 1 0 1 1 0 0 1 1 1 1 1 1 0 1 1 0 1 1 0 1 0 1 1 1 1 0 0 1 0 1 1 0 0]

2 of fold 5
accuracy score 0.7235772357723578
[0 1 1 1 0 1 1 0 1 1 1 1 1 1 1 1 0 1 1 1 0 0 1 1 0 1 0 1 1 1 0 1 1 1 1 0 1
 1 1 1 0 1 1 0 1 1 1 1 1 1 1 1 1 0 0 1 1 1 1 1 1 1 1 0 1 1 0 0 1 0 1 0 0 1
 1 1 1 1 1 

In [ ]:
import joblib

In [ ]:
joblib.dump(LogReg, "LogReg.joblib")
joblib.dump(DecTre, "DecTre.joblib")

['DecTre.joblib']

In [ ]:
X.columns

Index(['Credit_History', 'logLoanAmount', 'Gender_Female', 'Gender_Male',
       'Married_No', 'Married_Yes', 'Dependents_3', 'Dependents_0',
       'Dependents_1', 'Dependents_2', 'Education_Graduate',
       'Education_Not Graduate', 'Self_Employed_No', 'Self_Employed_Yes',
       'Property_Area_Rural', 'Property_Area_Semiurban', 'Property_Area_Urban',
       'Total_Income', 'logTotalIncome', 'EMI', 'Balance_Income'],
      dtype='object')